# VGG-19 First Pass

In [31]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 224, 224
model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [32]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 input_6 False
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_conv4 True
11 block3_pool True
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True


In [33]:
for layer in model.layers[:22]:
    layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


In [34]:
for i, layer in enumerate(model_final.layers):
    print(i, layer.name, layer.trainable)

0 input_6 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 False
18 block5_conv2 False
19 block5_conv3 False
20 block5_conv4 False
21 block5_pool False
22 flatten_5 True
23 dense_17 True
24 dropout_5 True
25 dense_18 True
26 dense_19 True


In [35]:
target_size=(224, 224) # change to (img_width, img_height) since it's already set at in the first cell of the notebook
seed = 123
batch_size = 32
epochs = 50
directory = 'data/train_images/'

train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    fill_mode = "nearest",
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3,
    rotation_range=30) # set validation split

train_generator = train_datagen.flow_from_directory(
    directory,
    target_size=target_size,
    batch_size=batch_size,
    seed=seed,
    class_mode="categorical",
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    directory, # same directory as training data
    target_size=target_size,
    batch_size=batch_size,
    seed=seed,
    class_mode="categorical",
    subset='validation') # set as validation data

Found 2931 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [36]:
# accidentally saved the weights as vgg16_1.h5 instead of vgg19_1.h5
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [37]:
# Train the model 
model_hist = model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = epochs,
    callbacks = [checkpoint, early])

Epoch 1/50
91/91 [==============================] - 1331s 15s/step - loss: 1.4197 - acc: 0.4839 - val_loss: 1.1443 - val_acc: 0.5270

Epoch 00001: val_acc improved from -inf to 0.52699, saving model to vgg16_1.h5
Epoch 2/50
91/91 [==============================] - 1310s 14s/step - loss: 1.2026 - acc: 0.5258 - val_loss: 1.1441 - val_acc: 0.5708

Epoch 00002: val_acc improved from 0.52699 to 0.57082, saving model to vgg16_1.h5
Epoch 3/50
91/91 [==============================] - 1301s 14s/step - loss: 1.1288 - acc: 0.5621 - val_loss: 1.0179 - val_acc: 0.6152

Epoch 00003: val_acc improved from 0.57082 to 0.61516, saving model to vgg16_1.h5
Epoch 4/50
91/91 [==============================] - 1299s 14s/step - loss: 1.1045 - acc: 0.5883 - val_loss: 1.0738 - val_acc: 0.5823

Epoch 00004: val_acc did not improve from 0.61516
Epoch 5/50
91/91 [==============================] - 1300s 14s/step - loss: 1.0738 - acc: 0.5985 - val_loss: 1.0131 - val_acc: 0.6180

Epoch 00005: val_acc improved from 0.

## Unfreezing all layers and continuing training

In [38]:
# accidentally saved the weights as vgg16_1.h5 instead of vgg19_1.h5 on first model. Continuing with vgg19_2.h5
checkpoint = ModelCheckpoint("vgg19_2.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [39]:
for i, layer in enumerate(model_final.layers):
    print(i, layer.name, layer.trainable)

0 input_6 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 False
18 block5_conv2 False
19 block5_conv3 False
20 block5_conv4 False
21 block5_pool False
22 flatten_5 True
23 dense_17 True
24 dropout_5 True
25 dense_18 True
26 dense_19 True


In [40]:
for layer in model.layers:
    layer.trainable = True

In [41]:
for i, layer in enumerate(model_final.layers):
    print(i, layer.name, layer.trainable)

0 input_6 True
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_conv4 True
11 block3_pool True
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True
22 flatten_5 True
23 dense_17 True
24 dropout_5 True
25 dense_18 True
26 dense_19 True


In [44]:
model_final.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

### Change generator to gapcv

In [57]:
epochs = 10

In [47]:
!pip install gapcv

     |████████████████████████████████| 26.6MB 8.5MB/s eta 0:00:01�                             | 1.8MB 8.5MB/s eta 0:00:035.0MB 8.5MB/s eta 0:00:03��███▌                      | 7.8MB 8.5MB/s eta 0:00:03.5MB/s eta 0:00:02�███████████████▎             | 15.2MB 8.5MB/s eta 0:00:02████████████████▊         | 18.8MB 8.5MB/s eta 0:00:01��█████████████████████████▏    | 22.5MB 8.5MB/s eta 0:00:01��████████▋ | 25.4MB 8.5MB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/fa/64/71/6be0efb1982b7ea259532e0db37c112d42156c06bb70ab9a2a
  Stored in directory: /home/jovyan/.cache/pip/wheels/b2/40/59/139d450e68847ef2f27d876d527b13389dac23df0f66526b5d
Successfully built gapcv imutils


In [48]:
from gapcv.vision import Images

In [49]:
images = Images('diabetic_retinopathy', directory, config=['resize=({},{})'.format(img_height, img_width), 'store', 'stream'])


In [51]:
images = Images(config=['stream'])
images.load('diabetic_retinopathy')

In [55]:
# split data set
images.split = 0.2
X_test, Y_test = images.test

# generator
images.minibatch = batch_size
gap_generator = images.minibatch

total_train_images = images.count - len(X_test)
n_classes = len(images.classes)

In [56]:
model_final.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [ ]:
# model.fit_generator(
#     generator=gap_generator,
#     validation_data=(X_test, Y_test),
#     steps_per_epoch=total_train_images // batch_size,
#     epochs=nb_epochs,
#     verbose=1
# )

In [ ]:
# Train the model 
model_hist2 = model_final.fit_generator(
    gap_generator,
    steps_per_epoch = total_train_images // batch_size,
    validation_data = (X_test, Y_test),
#     validation_steps = validation_generator.samples // batch_size,
    epochs = epochs,
    callbacks = [checkpoint, early])

Epoch 1/10
91/91 [==============================] - 3076s 34s/step - loss: 14.8284 - acc: 0.0800 - val_loss: 14.8207 - val_acc: 0.0805

Epoch 00001: val_acc improved from -inf to 0.08049, saving model to vgg19_2.h5
Epoch 2/10
91/91 [==============================] - 3088s 34s/step - loss: 14.8125 - acc: 0.0810 - val_loss: 14.8207 - val_acc: 0.0805

Epoch 00002: val_acc did not improve from 0.08049
Epoch 3/10
91/91 [==============================] - 3093s 34s/step - loss: 14.8560 - acc: 0.0783 - val_loss: 14.8207 - val_acc: 0.0805

Epoch 00003: val_acc did not improve from 0.08049
Epoch 4/10
91/91 [==============================] - 3083s 34s/step - loss: 14.7896 - acc: 0.0824 - val_loss: 14.8207 - val_acc: 0.0805

Epoch 00004: val_acc did not improve from 0.08049
Epoch 5/10
20/91 [=====>........................] - ETA: 37:18 - loss: 14.8085 - acc: 0.0813

In [46]:
# Train the model 
model_hist2 = model_final.fit_generator(
    gap_generator,
    steps_per_epoch = total_train_images // batch_size,
    validation_data = (X_test, Y_test),
#     validation_steps = validation_generator.samples // batch_size,
    epochs = epochs,
    callbacks = [checkpoint, early])

Epoch 1/10
 1/91 [..............................] - ETA: 1:05:31 - loss: 1.2555 - acc: 0.5312

KeyboardInterrupt: 

## Without Weights

In [10]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [11]:
img_width, img_height = 224, 224
model = applications.VGG19(weights = None, include_top=False, input_shape = (img_width, img_height, 3))

In [12]:
for i, layer in enumerate(model.layers):
    print(i, layer.name, layer.trainable)

0 input_2 False
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_conv4 True
11 block3_pool True
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True


In [13]:
# for layer in model.layers[:22]:
#     layer.trainable = False

#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(5, activation="softmax")(x)

# creating the final model 
model_final = Model(input = model.input, output = predictions)

# compile the model 
model_final.compile(loss='categorical_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


In [14]:
for i, layer in enumerate(model_final.layers):
    print(i, layer.name, layer.trainable)

0 input_2 False
1 block1_conv1 True
2 block1_conv2 True
3 block1_pool True
4 block2_conv1 True
5 block2_conv2 True
6 block2_pool True
7 block3_conv1 True
8 block3_conv2 True
9 block3_conv3 True
10 block3_conv4 True
11 block3_pool True
12 block4_conv1 True
13 block4_conv2 True
14 block4_conv3 True
15 block4_conv4 True
16 block4_pool True
17 block5_conv1 True
18 block5_conv2 True
19 block5_conv3 True
20 block5_conv4 True
21 block5_pool True
22 flatten_2 True
23 dense_4 True
24 dropout_2 True
25 dense_5 True
26 dense_6 True


In [15]:
target_size=(224, 224) # change to (img_width, img_height) since it's already set at in the first cell of the notebook
seed = 123
batch_size = 64
epochs = 50
directory = 'data/train_images/'

train_datagen = ImageDataGenerator(rescale=1./255,
#     shear_range=0.2,
#     horizontal_flip=True,
    validation_split=0.2)
#     fill_mode = "nearest",
#     zoom_range = 0.3,
#     width_shift_range = 0.3,
#     height_shift_range=0.3,
#     rotation_range=30) # set validation split

train_generator = train_datagen.flow_from_directory(
    directory,
    target_size=target_size,
    batch_size=batch_size,
    seed=seed,
    class_mode="categorical",
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    directory, # same directory as training data
    target_size=target_size,
    batch_size=batch_size,
    seed=seed,
    class_mode="categorical",
    subset='validation') # set as validation data

Found 2931 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [16]:
train_generator.class_indices

{'four': 0, 'one': 1, 'three': 2, 'two': 3, 'zero': 4}

In [17]:
# accidentally saved the weights as vgg16_1.h5 instead of vgg19_1.h5
checkpoint = ModelCheckpoint("vgg19_no_weights_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [18]:
# Train the model 
model_hist = model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = epochs,
    callbacks = [checkpoint, early])

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
45/45 [==============================] - 3178s 71s/step - loss: 1.5307 - acc: 0.4830 - val_loss: 1.4340 - val_acc: 0.4986

Epoch 00001: val_acc improved from -inf to 0.49858, saving model to vgg19_no_weights_1.h5
Epoch 2/50
45/45 [==============================] - 3106s 69s/step - loss: 1.3558 - acc: 0.4969 - val_loss: 1.3064 - val_acc: 0.4918

Epoch 00002: val_acc did not improve from 0.49858
Epoch 3/50
45/45 [==============================] - 3103s 69s/step - loss: 1.3086 - acc: 0.4868 - val_loss: 1.3053 - val_acc: 0.4903

Epoch 00003: val_acc did not improve from 0.49858
Epoch 4/50
45/45 [==============================] - 3105s 69s/step - loss: 1.3002 - acc: 0.4925 - val_loss: 1.2894 - val_acc: 0.4993

Epoch 00004: val_acc improved from 0.49858 to 0.49925, saving model to vgg19_no_weights_1.h5
Epoch 5/50
45/45 [==============================] - 3105s 69s/step - loss: 1.2911 - acc: 0.4999 - val_loss: 1.2634 - val_acc: 0.4933
